# Paired Path Browser

Interactive tool for exploring individual CRN-paired simulation paths.
Select a configuration and filter for interesting paths to compare **insured vs. uninsured**
trajectories for the same underlying random draws.

**Filters:**
- **Catastrophic** - Paths with >$2M single-year losses
- **Near-Ruin** - Bottom 1% of surviving simulations by final assets
- **Median** - Paths closest to the median final outcome
- **Best** - Top 1% by final assets
- **Worst Surviving** - Bottom 5% of survivors
- **Ruined** - Paths that went insolvent

**Note:** This browser loads pickle files on-demand from the results directory.
The first load for each configuration may be slow (~5-10s for 250K sim files).

**Prerequisites:** Run `1. process_vol_sim_results.ipynb` first to build `cache/dashboard_cache.pkl`.

In [ ]:
import pickle
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML
from pathlib import Path

In [ ]:
# Configuration - adjust this path to point to your simulation results
RESULTS_DIR = Path(r"D:\2026-01-31 Vol")

# Load dashboard cache
cache_path = Path("cache/dashboard_cache.pkl")
with open(cache_path, "rb") as f:
    cache = pickle.load(f)

configs = cache["configs"]
crn_pairs = cache["crn_pairs"]
param_values = cache["param_values"]

# Override results dir if cache has a different path
if not RESULTS_DIR.exists() and "results_dir" in cache:
    RESULTS_DIR = Path(cache["results_dir"])

print(f"Results dir: {RESULTS_DIR}")
print(f"Loaded {len(configs)} configs, {len(crn_pairs)} CRN pairs")

In [ ]:
# Pickle file cache - avoid re-loading the same file repeatedly
_pkl_cache = {}


def load_sim_results(config_key):
    """Load simulation results from pickle, with in-memory caching."""
    if config_key in _pkl_cache:
        return _pkl_cache[config_key]

    pkl_path = RESULTS_DIR / f"{config_key}.pkl"
    if not pkl_path.exists():
        # Try with whitespace variations in the filename
        candidates = list(RESULTS_DIR.glob(f"{config_key[:30]}*.pkl"))
        if candidates:
            pkl_path = candidates[0]
        else:
            raise FileNotFoundError(f"No pickle file found for: {config_key}")

    with open(pkl_path, "rb") as f:
        data = pickle.load(f)

    # Keep at most 4 loaded configs in memory (~2GB each)
    if len(_pkl_cache) >= 4:
        oldest = next(iter(_pkl_cache))
        del _pkl_cache[oldest]

    _pkl_cache[config_key] = data
    return data


def find_config_key(cap, atr, ded):
    """Find the config key matching the given insured parameters."""
    for key, cfg in configs.items():
        if (cfg.get("Cap") == cap
            and cfg.get("ATR") == atr
            and cfg.get("Ded") == ded
            and not cfg.get("NOINS")):
            return key
    return None


def find_noins_key(cap, atr):
    """Find the NOINS config key matching the given Cap and ATR."""
    for key, cfg in configs.items():
        if (cfg.get("Cap") == cap
            and cfg.get("ATR") == atr
            and cfg.get("NOINS")):
            return key
    return None


def fmt_amount(n):
    if n >= 1e9: return f"${n/1e9:.0f}B"
    if n >= 1e6: return f"${n/1e6:.0f}M"
    if n >= 1e3: return f"${n/1e3:.0f}K"
    return f"${n:.0f}"

In [ ]:
# Color scheme
INS_COLOR = "rgb(31,119,180)"
NI_COLOR = "rgb(255,127,14)"
LOSS_COLOR = "rgb(214,39,40)"
RECOVERY_COLOR = "rgb(44,160,44)"

# Create widgets
cap_dropdown = widgets.Dropdown(
    options=[(fmt_amount(v), v) for v in param_values["Cap"]],
    value=param_values["Cap"][0],
    description="Cap:",
    style={"description_width": "auto"},
)
atr_dropdown = widgets.Dropdown(
    options=[(str(v), v) for v in param_values["ATR"]],
    value=param_values["ATR"][1] if len(param_values["ATR"]) > 1 else param_values["ATR"][0],
    description="ATR:",
    style={"description_width": "auto"},
)
ded_dropdown = widgets.Dropdown(
    options=[(fmt_amount(v), v) for v in param_values["Ded"]],
    value=param_values["Ded"][1] if len(param_values["Ded"]) > 1 else param_values["Ded"][0],
    description="Deductible:",
    style={"description_width": "auto"},
)
filter_dropdown = widgets.Dropdown(
    options=["catastrophic", "near_ruin", "median", "best", "worst_surviving", "ruined"],
    value="catastrophic",
    description="Path filter:",
    style={"description_width": "auto"},
)
sim_id_dropdown = widgets.Dropdown(
    options=[],
    description="Sim ID:",
    style={"description_width": "auto"},
)

status_label = widgets.HTML(value="")
output_area = widgets.Output()

In [ ]:
def update_sim_ids(change=None):
    """Update the sim_id dropdown based on current config and filter."""
    cap = cap_dropdown.value
    atr = atr_dropdown.value
    ded = ded_dropdown.value
    filt = filter_dropdown.value

    ins_key = find_config_key(cap, atr, ded)
    if ins_key is None:
        sim_id_dropdown.options = []
        status_label.value = "<span style='color:red'>No config found</span>"
        return

    cfg = configs[ins_key]
    interesting = cfg.get("interesting_sims", {})
    sim_ids = interesting.get(filt, [])

    if not sim_ids:
        sim_id_dropdown.options = [("(none available)", -1)]
        status_label.value = f"<span style='color:orange'>No {filt} paths for this config</span>"
    else:
        sim_id_dropdown.options = [(f"#{sid} (final=${cfg['final_assets_qs'][int(sid / cfg['n_sims'] * 1000)] if sid < cfg['n_sims'] else 0:,.0f})", sid)
                                   if sid < len(cfg.get('final_assets_qs', [])) * cfg.get('n_sims', 1) / 1000
                                   else (f"#{sid}", sid)
                                   for sid in sim_ids[:50]]
        status_label.value = f"<span style='color:green'>{len(sim_ids)} {filt} paths available</span>"


def update_path_display(change=None):
    """Load and display the selected simulation path."""
    cap = cap_dropdown.value
    atr = atr_dropdown.value
    ded = ded_dropdown.value
    sim_id = sim_id_dropdown.value

    if sim_id is None or sim_id < 0:
        return

    ins_key = find_config_key(cap, atr, ded)
    noins_key = find_noins_key(cap, atr)

    if ins_key is None:
        with output_area:
            output_area.clear_output(wait=True)
            print("No insured config found.")
        return

    try:
        status_label.value = "<span style='color:blue'>Loading pickle files...</span>"
        ins_data = load_sim_results(ins_key)
        noins_data = load_sim_results(noins_key) if noins_key else None
        status_label.value = "<span style='color:green'>Data loaded</span>"
    except Exception as ex:
        with output_area:
            output_area.clear_output(wait=True)
            print(f"Error loading data: {ex}")
        status_label.value = f"<span style='color:red'>Load error: {ex}</span>"
        return

    n_years = ins_data.config.n_years
    years = np.arange(1, n_years + 1)
    cfg = configs[ins_key]
    initial_assets = cfg["initial_assets"]

    # Extract data for this sim_id
    ins_losses = ins_data.annual_losses[sim_id]
    ins_recoveries = ins_data.insurance_recoveries[sim_id]
    ins_retained = ins_data.retained_losses[sim_id]
    ins_final = ins_data.final_assets[sim_id]
    ins_growth = ins_data.growth_rates[sim_id]

    if noins_data is not None:
        ni_losses = noins_data.annual_losses[sim_id]
        ni_retained = noins_data.retained_losses[sim_id]
        ni_final = noins_data.final_assets[sim_id]
        ni_growth = noins_data.growth_rates[sim_id]
    else:
        ni_losses = ni_retained = None
        ni_final = ni_growth = None

    # Build visualization
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=[
            "Annual Ground-Up Losses",
            "Insurance Recoveries vs Retained Losses",
            "Cumulative Retained Losses",
            "Approximate Wealth Trajectories",
            "Annual Loss Decomposition (Insured)",
            "Loss Severity Comparison",
        ],
        vertical_spacing=0.10,
        horizontal_spacing=0.08,
    )

    # ===== Panel 1: Annual Ground-Up Losses =====
    fig.add_trace(go.Bar(
        x=years, y=ins_losses,
        name="Insured Losses", marker_color=INS_COLOR, opacity=0.7,
    ), row=1, col=1)

    if ni_losses is not None:
        fig.add_trace(go.Bar(
            x=years, y=ni_losses,
            name="Uninsured Losses", marker_color=NI_COLOR, opacity=0.7,
        ), row=1, col=1)

    fig.update_xaxes(title_text="Year", row=1, col=1)
    fig.update_yaxes(title_text="Ground-Up Loss ($)", row=1, col=1)

    # ===== Panel 2: Recoveries vs Retained (insured path) =====
    fig.add_trace(go.Bar(
        x=years, y=ins_retained,
        name="Retained (Insured)", marker_color=LOSS_COLOR, opacity=0.7,
    ), row=1, col=2)
    fig.add_trace(go.Bar(
        x=years, y=ins_recoveries,
        name="Insurance Recovery", marker_color=RECOVERY_COLOR, opacity=0.7,
    ), row=1, col=2)

    fig.update_xaxes(title_text="Year", row=1, col=2)
    fig.update_yaxes(title_text="Amount ($)", row=1, col=2)
    fig.update_layout(barmode="group")

    # ===== Panel 3: Cumulative Retained Losses =====
    cum_ins_retained = np.cumsum(ins_retained)
    fig.add_trace(go.Scatter(
        x=years, y=cum_ins_retained,
        line=dict(color=INS_COLOR, width=2.5),
        name="Cum. Retained (Insured)",
    ), row=2, col=1)

    if ni_retained is not None:
        cum_ni_retained = np.cumsum(ni_retained)
        fig.add_trace(go.Scatter(
            x=years, y=cum_ni_retained,
            line=dict(color=NI_COLOR, width=2.5, dash="dash"),
            name="Cum. Retained (Uninsured)",
        ), row=2, col=1)

        # Savings
        savings = cum_ni_retained - cum_ins_retained
        fig.add_trace(go.Scatter(
            x=years, y=savings,
            line=dict(color=RECOVERY_COLOR, width=1.5, dash="dot"),
            name="Cumulative Savings",
            fill="tozeroy", fillcolor="rgba(44,160,44,0.1)",
        ), row=2, col=1)

    fig.update_xaxes(title_text="Year", row=2, col=1)
    fig.update_yaxes(title_text="Cumulative Retained Loss ($)", row=2, col=1)

    # ===== Panel 4: Approximate Wealth Trajectories =====
    # Exponential interpolation from growth rate
    full_years = np.arange(n_years + 1)
    ins_wealth = initial_assets * np.exp(ins_growth * full_years)
    fig.add_trace(go.Scatter(
        x=full_years, y=ins_wealth,
        line=dict(color=INS_COLOR, width=2.5),
        name="Insured Wealth (approx)",
    ), row=2, col=2)

    if ni_growth is not None:
        ni_wealth = initial_assets * np.exp(ni_growth * full_years)
        fig.add_trace(go.Scatter(
            x=full_years, y=ni_wealth,
            line=dict(color=NI_COLOR, width=2.5, dash="dash"),
            name="Uninsured Wealth (approx)",
        ), row=2, col=2)

    fig.add_hline(y=10_000, line_dash="dot", line_color="red",
                  annotation_text="Ruin", row=2, col=2)
    fig.update_yaxes(type="log", title_text="Assets ($)", row=2, col=2)
    fig.update_xaxes(title_text="Year", row=2, col=2)

    # ===== Panel 5: Loss Decomposition (stacked bar) =====
    fig.add_trace(go.Bar(
        x=years, y=ins_retained,
        name="Retained by Company", marker_color=LOSS_COLOR, opacity=0.8,
    ), row=3, col=1)
    fig.add_trace(go.Bar(
        x=years, y=ins_recoveries,
        name="Paid by Insurer", marker_color=RECOVERY_COLOR, opacity=0.8,
    ), row=3, col=1)

    fig.update_xaxes(title_text="Year", row=3, col=1)
    fig.update_yaxes(title_text="Loss Amount ($)", row=3, col=1)

    # ===== Panel 6: Loss Severity Comparison =====
    # Compare years where large losses occurred
    large_loss_mask = ins_losses > np.percentile(ins_losses[ins_losses > 0], 75) if (ins_losses > 0).sum() > 4 else ins_losses > 0
    large_years = years[large_loss_mask]

    if len(large_years) > 0:
        fig.add_trace(go.Scatter(
            x=large_years, y=ins_losses[large_loss_mask],
            mode="markers", marker=dict(size=10, color=INS_COLOR),
            name="Large Losses (Insured)",
        ), row=3, col=2)
        fig.add_trace(go.Scatter(
            x=large_years, y=ins_retained[large_loss_mask],
            mode="markers", marker=dict(size=10, color=LOSS_COLOR, symbol="diamond"),
            name="Retained Portion",
        ), row=3, col=2)
        fig.add_trace(go.Scatter(
            x=large_years, y=ins_recoveries[large_loss_mask],
            mode="markers", marker=dict(size=10, color=RECOVERY_COLOR, symbol="triangle-up"),
            name="Insurance Recovery",
        ), row=3, col=2)

    fig.update_xaxes(title_text="Year", row=3, col=2)
    fig.update_yaxes(title_text="Amount ($)", row=3, col=2)

    # Layout
    title = (f"Sim #{sim_id} | Cap={fmt_amount(cap)} | ATR={atr} | Ded={fmt_amount(ded)}")
    fig.update_layout(
        height=1100, width=1300,
        title_text=f"Paired Path Browser: {title}",
        title_font_size=16,
        showlegend=True,
        legend=dict(font=dict(size=9), orientation="h", yanchor="bottom", y=-0.08),
        margin=dict(t=80, b=80),
    )

    # Summary metrics table
    rows_html = []
    metrics = [
        ("Growth Rate", f"{ins_growth:.5f}", f"{ni_growth:.5f}" if ni_growth is not None else "-"),
        ("Final Assets", fmt_amount(ins_final), fmt_amount(ni_final) if ni_final is not None else "-"),
        ("Total Losses", fmt_amount(ins_losses.sum()), fmt_amount(ni_losses.sum()) if ni_losses is not None else "-"),
        ("Total Retained", fmt_amount(ins_retained.sum()), fmt_amount(ni_retained.sum()) if ni_retained is not None else "-"),
        ("Total Recoveries", fmt_amount(ins_recoveries.sum()), "-"),
        ("Max Single-Year Loss", fmt_amount(ins_losses.max()), fmt_amount(ni_losses.max()) if ni_losses is not None else "-"),
        ("Years With Losses", str((ins_losses > 0).sum()), str((ni_losses > 0).sum()) if ni_losses is not None else "-"),
        ("Wealth Multiple (50yr)", f"{ins_final / initial_assets:.2f}x",
         f"{ni_final / initial_assets:.2f}x" if ni_final is not None else "-"),
    ]

    for label, v_ins, v_ni in metrics:
        rows_html.append(
            f"<tr><td style='padding:3px 10px'><b>{label}</b></td>"
            f"<td style='padding:3px 10px; text-align:right'>{v_ins}</td>"
            f"<td style='padding:3px 10px; text-align:right'>{v_ni}</td></tr>"
        )

    table_html = (
        "<table style='border-collapse:collapse; font-family:monospace; font-size:12px; margin-top:10px'>"
        "<tr style='border-bottom:2px solid #333'>"
        "<th style='padding:3px 10px; text-align:left'>Metric</th>"
        "<th style='padding:3px 10px; text-align:right; color:#1f77b4'>Insured</th>"
        "<th style='padding:3px 10px; text-align:right; color:#ff7f0e'>Uninsured</th></tr>"
        + "\n".join(rows_html)
        + "</table>"
    )

    with output_area:
        output_area.clear_output(wait=True)
        fig.show()
        display(HTML(table_html))

In [ ]:
# Wire up callbacks
# Config/filter changes -> update sim_id list
cap_dropdown.observe(update_sim_ids, names="value")
atr_dropdown.observe(update_sim_ids, names="value")
ded_dropdown.observe(update_sim_ids, names="value")
filter_dropdown.observe(update_sim_ids, names="value")

# Sim ID change -> update path display
sim_id_dropdown.observe(update_path_display, names="value")

# Layout
config_row = widgets.HBox(
    [cap_dropdown, atr_dropdown, ded_dropdown],
    layout=widgets.Layout(gap="15px"),
)
filter_row = widgets.HBox(
    [filter_dropdown, sim_id_dropdown, status_label],
    layout=widgets.Layout(gap="15px"),
)

display(config_row)
display(filter_row)
display(output_area)

# Initialize
update_sim_ids()

## Multi-Path Overlay

Compare multiple paths from the same filter category overlaid on a single chart.
Useful for identifying patterns in catastrophic loss paths or near-ruin scenarios.

In [ ]:
def show_multi_path_overlay(n_paths=10):
    """Overlay multiple filtered paths for pattern recognition."""
    cap = cap_dropdown.value
    atr = atr_dropdown.value
    ded = ded_dropdown.value
    filt = filter_dropdown.value

    ins_key = find_config_key(cap, atr, ded)
    noins_key = find_noins_key(cap, atr)

    if ins_key is None:
        print("No config found.")
        return

    cfg = configs[ins_key]
    sim_ids = cfg.get("interesting_sims", {}).get(filt, [])
    if not sim_ids:
        print(f"No {filt} paths available.")
        return

    sim_ids = sim_ids[:n_paths]

    try:
        ins_data = load_sim_results(ins_key)
        noins_data = load_sim_results(noins_key) if noins_key else None
    except Exception as ex:
        print(f"Error loading: {ex}")
        return

    n_years = ins_data.config.n_years
    years = np.arange(1, n_years + 1)

    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=[
            f"Cumulative Retained Losses ({filt}, {len(sim_ids)} paths)",
            f"Annual Losses ({filt})",
        ],
    )

    for i, sid in enumerate(sim_ids):
        alpha = 0.5
        show_legend = (i == 0)

        # Insured cumulative retained
        cum_ret = np.cumsum(ins_data.retained_losses[sid])
        fig.add_trace(go.Scatter(
            x=years, y=cum_ret,
            line=dict(color=INS_COLOR, width=1),
            opacity=alpha,
            name="Insured" if show_legend else None,
            showlegend=show_legend,
            legendgroup="ins",
        ), row=1, col=1)

        if noins_data is not None:
            cum_ret_ni = np.cumsum(noins_data.retained_losses[sid])
            fig.add_trace(go.Scatter(
                x=years, y=cum_ret_ni,
                line=dict(color=NI_COLOR, width=1, dash="dash"),
                opacity=alpha,
                name="Uninsured" if show_legend else None,
                showlegend=show_legend,
                legendgroup="noins",
            ), row=1, col=1)

        # Annual losses
        fig.add_trace(go.Scatter(
            x=years, y=ins_data.annual_losses[sid],
            line=dict(color=INS_COLOR, width=1),
            opacity=alpha, showlegend=False, legendgroup="ins",
        ), row=1, col=2)

    fig.update_xaxes(title_text="Year", row=1, col=1)
    fig.update_xaxes(title_text="Year", row=1, col=2)
    fig.update_yaxes(title_text="Cumulative Retained Loss ($)", row=1, col=1)
    fig.update_yaxes(title_text="Annual Loss ($)", row=1, col=2)

    fig.update_layout(
        height=450, width=1300,
        title_text=f"Multi-Path Overlay: {filt} | Cap={fmt_amount(cap)} | ATR={atr} | Ded={fmt_amount(ded)}",
        title_font_size=14,
    )
    fig.show()

    # Summary stats for these paths
    finals = ins_data.final_assets[sim_ids]
    growths = ins_data.growth_rates[sim_ids]
    print(f"\n{filt.title()} Paths Summary ({len(sim_ids)} paths):")
    print(f"  Growth rate: mean={growths.mean():.5f}, std={growths.std():.5f}")
    print(f"  Final assets: mean={fmt_amount(finals.mean())}, median={fmt_amount(np.median(finals))}")
    print(f"  Max single-year loss: {fmt_amount(ins_data.annual_losses[sim_ids].max())}")


show_multi_path_overlay()